In [19]:
import re
import pandas as pd

def parse_query_plans(file_path:str, test_name:str, baseline:float) -> pd.DataFrame:
    """
    Parse PostgreSQL EXPLAIN ANALYZE output and extract key metrics.
    
    Parameters:
    file_path: Path to the text file containing query plans
    test_name: Name for this test run (e.g., "Single Table", "Split Table")
    baseline
    
    Returns:
    DataFrame with columns: Test, Factors, Scan_Type, Planning_Time_ms, Execution_Time_ms
    """
    
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Split into individual query plans (separated by rows count indicator)
    # Each plan ends with "(X rows)" pattern
    plans = re.split(r'\(\d+ rows?\)\n\n', content)
    
    results = []
    
    for idx, plan in enumerate(plans):
        if not plan.strip():
            continue
            
        # Extract factors from Filter line
        # Looking for pattern like: factors @> '{5}' or factors @> '{11,13}'
        factors_match = re.search(r"factors @> '\{([^}]+)\}'", plan)
        if not factors_match:
            continue
        factors = factors_match.group(1)
        num_factors = len(factors.split(','))
        
        # Extract scan type
        # Looking for lines like "-> Parallel Seq Scan on" or "-> Index Scan using"
        scan_match = re.search(r'->\s+([^(]+?)\s+on', plan)
        if not scan_match:
            continue
        scan_type = scan_match.group(1).strip()
        
        # Extract planning time
        planning_match = re.search(r'Planning Time:\s+([\d.]+)\s+ms', plan)
        if not planning_match:
            continue
        planning_time = float(planning_match.group(1))
        
        # Extract execution time
        execution_match = re.search(r'Execution Time:\s+([\d.]+)\s+ms', plan)
        if not execution_match:
            continue
        execution_time = float(execution_match.group(1))
        
        results.append({
            'test': test_name,
            'test_number': idx + 1,
            'factors': factors,
            'num_of_factors': num_factors,
            'scan_type': scan_type,
            'baseline_ms': baseline,
            'planning_time_ms': planning_time,
            'execution_time_ms': execution_time,
            'total_time': planning_time + execution_time,
            'time_improvement': baseline - (planning_time + execution_time),
            'improvement_percent': (baseline - (planning_time + execution_time)) / baseline
        })
    
    return pd.DataFrame(results)

In [20]:
baseline = 265000
one_table = parse_query_plans('one_table_results_1.txt', 'Single Table', baseline)
multi_table = parse_query_plans('one_table_results_2.txt', 'Multi Table', baseline)

print("Single Table")
display(one_table)

print()
print("Multi Table")
display(multi_table)

Single Table


,test,test_number,factors,num_of_factors,scan_type,baseline_ms,planning_time_ms,execution_time_ms,total_time,time_improvement,improvement_percent
0,Single Table,1,5,1,Parallel Seq Scan,265000,1.140,17512.988,17514.128,247485.872,0.933909
1,Single Table,2,14348909,1,Parallel Bitmap Heap Scan,265000,0.096,26.497,26.593,264973.407,0.999900
2,Single Table,3,"11,13",2,Parallel Seq Scan,265000,0.074,17714.209,17714.283,247285.717,0.933154
3,Single Table,4,"21871,21881",2,Bitmap Index Scan,265000,0.093,1.697,1.790,264998.210,0.999993
4,Single Table,5,"11,13,17",3,Parallel Bitmap Heap Scan,265000,0.052,10063.930,10063.982,254936.018,0.962023
5,Single Table,6,"271,277,281",3,Bitmap Index Scan,265000,0.160,992.903,993.063,264006.937,0.996253
6,Single Table,7,"29,31,37,41",4,Parallel Bitmap Heap Scan,265000,0.094,7366.357,7366.451,257633.549,0.972202
7,Single Table,8,"107,109,113,127",4,Bitmap Index Scan,265000,0.096,779.444,779.540,264220.460,0.997058
8,Single Table,9,"53,59,61,67,71",5,Bitmap Index Scan,265000,0.090,1697.697,1697.787,263302.213,0.993593
9,Single Table,10,"79,83,89,97,101",5,Bitmap Index Scan,265000,0.086,990.881,990.967,264009.033,0.996261



Multi Table


,test,test_number,factors,num_of_factors,scan_type,baseline_ms,planning_time_ms,execution_time_ms,total_time,time_improvement,improvement_percent
0,Multi Table,1,5,1,Seq Scan,265000,5.901,34129.495,34135.396,230864.604,0.871187
1,Multi Table,2,14348909,1,Seq Scan,265000,0.609,15557.104,15557.713,249442.287,0.941292
2,Multi Table,3,"11,13",2,Parallel Seq Scan,265000,0.342,17722.589,17722.931,247277.069,0.933121
3,Multi Table,4,"21871,21881",2,Seq Scan,265000,0.339,15446.157,15446.496,249553.504,0.941711
4,Multi Table,5,"11,13,17",3,Seq Scan,265000,0.378,16088.577,16088.955,248911.045,0.939287
5,Multi Table,6,"271,277,281",3,Seq Scan,265000,0.362,15651.678,15652.040,249347.960,0.940936
6,Multi Table,7,"29,31,37,41",4,Seq Scan,265000,0.346,16356.950,16357.296,248642.704,0.938274
7,Multi Table,8,"107,109,113,127",4,Seq Scan,265000,0.344,15353.657,15354.001,249645.999,0.942060
8,Multi Table,9,"53,59,61,67,71",5,Seq Scan,265000,0.324,15417.340,15417.664,249582.336,0.941820
9,Multi Table,10,"79,83,89,97,101",5,Seq Scan,265000,0.322,15610.705,15611.027,249388.973,0.941090


In [28]:
results = pd.concat([one_table, multi_table]).sort_values(['test_number', 'test'], ascending=[True, False])

results

,test,test_number,factors,num_of_factors,scan_type,baseline_ms,planning_time_ms,execution_time_ms,total_time,time_improvement,improvement_percent
0,Single Table,1,5,1,Parallel Seq Scan,265000,1.140,17512.988,17514.128,247485.872,0.933909
0,Multi Table,1,5,1,Seq Scan,265000,5.901,34129.495,34135.396,230864.604,0.871187
1,Single Table,2,14348909,1,Parallel Bitmap Heap Scan,265000,0.096,26.497,26.593,264973.407,0.999900
1,Multi Table,2,14348909,1,Seq Scan,265000,0.609,15557.104,15557.713,249442.287,0.941292
2,Single Table,3,"11,13",2,Parallel Seq Scan,265000,0.074,17714.209,17714.283,247285.717,0.933154
2,Multi Table,3,"11,13",2,Parallel Seq Scan,265000,0.342,17722.589,17722.931,247277.069,0.933121
3,Single Table,4,"21871,21881",2,Bitmap Index Scan,265000,0.093,1.697,1.790,264998.210,0.999993
3,Multi Table,4,"21871,21881",2,Seq Scan,265000,0.339,15446.157,15446.496,249553.504,0.941711
4,Single Table,5,"11,13,17",3,Parallel Bitmap Heap Scan,265000,0.052,10063.930,10063.982,254936.018,0.962023
4,Multi Table,5,"11,13,17",3,Seq Scan,265000,0.378,16088.577,16088.955,248911.045,0.939287


In [29]:
results.to_csv('results_combined.csv', index=False)

In [30]:
results.columns

Index(['test', 'test_number', 'factors', 'num_of_factors', 'scan_type',
       'baseline_ms', 'planning_time_ms', 'execution_time_ms', 'total_time',
       'time_improvement', 'improvement_percent'],
      dtype='object')